In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gmplot
import torch
from torch import nn
from tqdm import tqdm
from tqdm import trange
import time
from tqdm import trange
import h3
from collections import OrderedDict
from sklearn.preprocessing import  OneHotEncoder
import move_fromd2l as d2l

def geo_t_h3(data):
    h3_list =OrderedDict()

    for i in data:
        a = h3.geo_to_h3(i[0], i[1], 10)
        h3_list.setdefault(a)
    return h3_list
def h3_t_geo(data):
    new_list = []
    for i in data:
        i =h3.h3_to_geo(i)
        new_list.append(i)
    return new_list

# draw(,130,10)
# list,文件名，放大率
def draw(list,number,b):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    lat = torch.stack(lat)
    lng =torch.stack(lng)
    gmap = gmplot.GoogleMapPlotter(lat[0], lng[0], b)
    gmap.plot(lat, lng,color='r',lw=10)  #描绘轨迹点
    gmap.draw("user{}.html".format(number))   #显示图
    print("over")
# 这个提取出来有5个维度
def dataset(user,start,end,step):
    # user:第几个用户
    # filenumber:取前面几天数据
    # step：隔了多少步取一次
    # userdata =  '.\\Geolife Trajectories 1.3\\Data\\' + user + '\\Trajectory\\'
    # print(os.listdir('..'))路径老大难，主义当前文件位置
    userdata='../Geolife Trajectories 1.3/Data/'+user+'/Trajectory/'
    print(userdata)
    filelist = os.listdir(userdata)  #返回指定路径下所有文件和文件夹的名字，并存放于一个列表中
    filelist.sort()
    names = ['lat','lng','zero','alt','days','date','time']
    df_list = [# f为文件索引号，header为列数，names为列表列名，index_col为行索引的列编号或列名
        pd.read_csv(userdata + f,header=6,names=names,index_col=False)
    #隔行读取
        for f in filelist[start:end]]
    df = pd.concat(df_list, ignore_index=True) #表格列字段不同的表合并
    df.drop(['zero', 'days'], axis=1, inplace=True) #drop函数默认删除行，列需要加axis = 1
    df_min = df.iloc[::step, :]
    return df_min
# 这个提取出来有2个维度
def synthetic_data(df_min):
    a =df_min['lat'].tolist()
    b = df_min['lng'].tolist()
    a = torch.tensor(a,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    b = torch.tensor(b,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    features = torch.concat([a,b],1)
    return features
# 将(lat,lng)变成哈希值
def generate_h3_list(data):
    alist = geo_t_h3(data)
    LIST = list(alist.keys())
    return np.array(LIST)


# 这个提取出来有5个维度
train_dataset = dataset("006",0,20,60)
test_dataset =  dataset("006",20,25,60)

# 这个提取出来有2个维度，只保留经纬度了
train_data = synthetic_data(train_dataset)
test_data = synthetic_data(test_dataset)
# concat()
all_data =torch.concat([train_data,test_data],0)



Train_h3_list  = generate_h3_list(train_data)
# print(Train_h3_list)这个相当于是h3
TESETLIST  = generate_h3_list(test_data)
all_data_h3_list = generate_h3_list(all_data)
# 这个all_data_h3_list可以先shuffle一下


# 可以通过idnex找到对应的h3
vocab_reverse  = dict(enumerate(all_data_h3_list))
# 可以通过h3编码找到对应的index
vocab ={h3:i for i ,h3 in vocab_reverse.items()}

encoder = OneHotEncoder(sparse=False).fit(all_data_h3_list.reshape(-1,1))

#y是一个列表，k是步长
def dataloader(y,k):
    data =[]
    for i in range(len(y)-k+1):
        indata = y[i:i+k]
        outdata = y[i+k:i+k+1]
        data.append((indata,outdata))
    return data
train_dataloader = dataloader(Train_h3_list,10)
test_dataloader =dataloader(test_data,10)




# 提升：
# 1。如何生成能够进行批训练的dataloader
# 2。修改学习率,步子长短,取样的时间达到更好的效果
# 3。现在的预测只能预测下一个点，如果想预测轨迹，应当把预测的点也当作数据输入模型
# 4.









../Geolife Trajectories 1.3/Data/006/Trajectory/
../Geolife Trajectories 1.3/Data/006/Trajectory/
['8a31aa50e857fff' '8a31aa50e847fff' '8a31aa50eb07fff' '8a31aa5089affff'
 '8a31aa50899ffff' '8a31aa5088c7fff' '8a31aa508baffff' '8a31aa508a17fff'
 '8a31aa50b89ffff' '8a31aa50b6affff' '8a31aa420b27fff' '8a31aa4202a7fff'
 '8a31aa423887fff' '8a31aa4237affff' '8a31aa40406ffff' '8a31aa40551ffff'
 '8a31aa405217fff' '8a31aa42a457fff' '8a31aa40c927fff' '8a31aa40d927fff'
 '8a31aa508307fff' '8a31aa508117fff' '8a31aa5080a7fff' '8a31aa5080b7fff'
 '8a31aa50e26ffff' '8a31aa50e277fff' '8a31aa50e0c7fff' '8a31aa50e50ffff'
 '8a31aa50339ffff' '8a31aa51d137fff' '8a31aa51d7affff' '8a31aa519437fff'
 '8a31aa51ba87fff' '8a31aa51b287fff' '8a31aa43544ffff' '8a31aa431597fff'
 '8a31aa41599ffff' '8a31aa406657fff' '8a31aa402d77fff' '8a31aa402d5ffff'
 '8a31aa402c0ffff' '8a31aa402957fff' '8a31aa40062ffff' '8a31aa400157fff'
 '8a31aa401c6ffff' '8a31aa40181ffff' '8a31aa40530ffff' '8a31aa405a27fff'
 '8a31aa42e427fff' '8a31aa

In [14]:
class RNN(nn.Module):
    def __init__(self,vocab) -> None:
        super().__init__()
        self.vocab = vocab
        self.model =nn.LSTM(
            input_size=vocab,
            hidden_size=320,
            batch_first=True,
            bias=True
        )
        self.output = nn.Sequential(
            nn.Linear(320,vocab),
            nn.ReLU()
        )

    def forward(self,x):
        r_out,(h_out,c_out) = self.model(x.view(1,10,self.vocab) ,None)
        outdata = self.output(r_out[:,-1,:])
        return outdata
net = RNN(len(all_data_h3_list))
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1,momentum=0.8)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_function = nn.CrossEntropyLoss()


In [ ]:
for epoch in trange(1000):
    loss_ =0
    for i , k in train_dataloader[:-1]:
        optimizer.zero_grad()
        a = encoder.transform(i.reshape(-1,1))
        k = encoder.transform(k.reshape(-1,1))
        a =torch.tensor(a).to(torch.float32)
        k =torch.tensor(k).to(torch.float32)
        loss = loss_function(net(a),k)
        loss.backward()
        optimizer.step()
        loss_ = loss_+loss
    print(loss_)

  0%|          | 1/1000 [00:06<1:41:48,  6.12s/it]

tensor(1584.8191, grad_fn=<AddBackward0>)


  0%|          | 2/1000 [00:11<1:33:17,  5.61s/it]

tensor(1585.1207, grad_fn=<AddBackward0>)


  0%|          | 3/1000 [00:17<1:35:06,  5.72s/it]

...

tensor(1298.5833, grad_fn=<AddBackward0>)


 20%|██        | 203/1000 [19:49<1:15:57,  5.72s/it]

tensor(1298.5831, grad_fn=<AddBackward0>)


 20%|██        | 204/1000 [19:55<1:16:13,  5.75s/it]

tensor(1298.5829, grad_fn=<AddBackward0>)


In [ ]:

test_dataloader = dataloader(TESETLIST,10)

# 能否根据当前的预测取进行下一步的预测
lat_l =[]
for i , k in test_dataloader[:-1]:
        a = encoder.transform(i.reshape(-1,1))
        k = encoder.transform(k.reshape(-1,1))
        a =torch.tensor(a).to(torch.float32)
        k =torch.tensor(k).to(torch.float32)
        out = net(a)
        out = out.detach().numpy()
        index = np.argmax(out)
        print(index)
        d =vocab_reverse.get(index)
        f = h3.h3_to_geo(str(d))
        lat_l.append(f)

draw(lat_l,150,10)



